* The following code is modified based on the original torchcv project. We are going to use PASACAL VOC12 as dataset. 

In [1]:
import os
import random
import argparse
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms

from __future__ import print_function
from torchcv.models.fpnssd import FPNSSD512
from torchcv.models.fpnssd import FPNSSDBoxCoder

from torchcv.loss import SSDLoss
from torchcv.datasets import ListDataset
from torchcv.transforms import resize, random_flip, random_paste, random_crop, random_distort

In [2]:
#parser = argparse.ArgumentParser(description='PyTorch FPNSSD Training')
#parser.add_argument('--lr', default=1e-3, type=float, help='learning rate')
#parser.add_argument('--resume', '-r', action='store_true', help='resume from checkpoint')
#parser.add_argument('--model', default='./model/fpn.pth', type=str, help='initialized model path')
#parser.add_argument('--checkpoint', default='./checkpoint/fpn_ckpt.pth', type=str, help='checkpoint path')
#args = parser.parse_args()
args = { "lr":1e-3,
        "resume":False,
       "model":"./model/fpn.pth",
       "checkpoint":"./checkpoint/fpn_ckpt.pth"}

In [3]:
print('==> Preparing dataset..')
img_size = 512
box_coder = FPNSSDBoxCoder()

def transform_train(img, boxes, labels):
    img = random_distort(img)
    if random.random() < 0.5:
        img, boxes = random_paste(img, boxes, max_ratio=4, fill=(123,116,103))
    img, boxes, labels = random_crop(img, boxes, labels)
    img, boxes = resize(img, boxes, size=(img_size,img_size), random_interpolation=True)
    img, boxes = random_flip(img, boxes)
    img = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.485,0.456,0.406),(0.229,0.224,0.225))
    ])(img)
    boxes, labels = box_coder.encode(boxes, labels)
    return img, boxes, labels

trainset = ListDataset(root='./dataset/VOC12-HR/',    \
                       list_file='./dataset/voc12_train.txt', \
                       transform=transform_train)

def transform_test(img, boxes, labels):
    img, boxes = resize(img, boxes, size=(img_size,img_size))
    img = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.485,0.456,0.406),(0.229,0.224,0.225))
    ])(img)
    boxes, labels = box_coder.encode(boxes, labels)
    return img, boxes, labels

testset = ListDataset(root='./dataset/VOC12-HR/',  \
                      list_file='./dataset/voc12_train.txt', \
                        transform=transform_test)

==> Preparing dataset..


In [4]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=8, shuffle=True, num_workers=8)
testloader = torch.utils.data.DataLoader(testset, batch_size=8, shuffle=False, num_workers=8)

# Model
print('==> Building model..')

device = 'cuda' if torch.cuda.is_available() else 'cpu'

print("device:",device)

net = FPNSSD512(num_classes=21).to(device)
#net.load_state_dict(torch.load(args.model))
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

best_loss = float('inf')  # best test loss
start_epoch = 0  # start from epoch 0 or last epoch

if args["resume"]:
    print('==> Resuming from checkpoint..')
    checkpoint = torch.load(args["checkpoint"])
    net.load_state_dict(checkpoint['net'])
    best_loss = checkpoint['loss']
    start_epoch = checkpoint['epoch']

criterion = SSDLoss(num_classes=21)
optimizer = optim.SGD(net.parameters(), lr=args['lr'], momentum=0.9, weight_decay=1e-4)
#import pdb; pdb.set_trace()
# Training
def train(epoch):
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    
    for batch_idx, (inputs, loc_targets, cls_targets) in enumerate(trainloader):
        inputs = inputs.to(device)
        loc_targets = loc_targets.to(device)
        cls_targets = cls_targets.to(device)

        optimizer.zero_grad()
        loc_preds, cls_preds = net(inputs)
        loss = criterion(loc_preds, loc_targets, cls_preds, cls_targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        print('train_loss: %.3f | avg_loss: %.3f [%d/%d]'
              % (loss.item(), train_loss/(batch_idx+1), batch_idx+1, len(trainloader)))
        break;

def test(epoch):
    print('\nTest')
    net.eval()
    test_loss = 0
    with torch.no_grad():
        for batch_idx, (inputs, loc_targets, cls_targets) in enumerate(testloader):
            inputs = inputs.to(device)
            loc_targets = loc_targets.to(device)
            cls_targets = cls_targets.to(device)

            loc_preds, cls_preds = net(inputs)
            loss = criterion(loc_preds, loc_targets, cls_preds, cls_targets)
            test_loss += loss.item()
            print('test_loss: %.3f | avg_loss: %.3f [%d/%d]'
                  % (loss.item(), test_loss/(batch_idx+1), batch_idx+1, len(testloader)))
            break;

    # Save checkpoint
    global best_loss
    test_loss /= len(testloader)
    if test_loss < best_loss:
        print('Saving..')
        state = {
            'net': net.state_dict(),
            'loss': test_loss,
            'epoch': epoch,
        }
        if not os.path.isdir(os.path.dirname(args['checkpoint'])):
            os.mkdir(os.path.dirname(args['checkpoint']))
        torch.save(state, args['checkpoint'])
        best_loss = test_loss


for epoch in range(start_epoch, start_epoch+200):
    train(epoch)
    test(epoch)


==> Building model..
device: cuda

Epoch: 0
loc_loss: 0.098 | cls_loss: 12.313 | train_loss: 12.412 | avg_loss: 12.412 [1/2141]

Test
loc_loss: 0.089 | cls_loss: 12.248 | test_loss: 12.337 | avg_loss: 12.337 [1/2141]
Saving..

Epoch: 1
loc_loss: 0.093 | cls_loss: 12.283 | train_loss: 12.376 | avg_loss: 12.376 [1/2141]

Test
loc_loss: 0.089 | cls_loss: 12.240 | test_loss: 12.329 | avg_loss: 12.329 [1/2141]
Saving..

Epoch: 2
loc_loss: 0.092 | cls_loss: 12.278 | train_loss: 12.371 | avg_loss: 12.371 [1/2141]

Test
loc_loss: 0.089 | cls_loss: 12.233 | test_loss: 12.322 | avg_loss: 12.322 [1/2141]
Saving..


Exception ignored in: <bound method _DataLoaderIter.__del__ of <torch.utils.data.dataloader._DataLoaderIter object at 0x7f33be230748>>
Traceback (most recent call last):
  File "/home/shared/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 349, in __del__
    self._shutdown_workers()
  File "/home/shared/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 328, in _shutdown_workers
    self.worker_result_queue.get()
  File "/home/shared/anaconda3/lib/python3.6/multiprocessing/queues.py", line 337, in get
    return _ForkingPickler.loads(res)
  File "/home/shared/anaconda3/lib/python3.6/site-packages/torch/multiprocessing/reductions.py", line 70, in rebuild_storage_fd
    fd = df.detach()
  File "/home/shared/anaconda3/lib/python3.6/multiprocessing/resource_sharer.py", line 57, in detach
    with _resource_sharer.get_connection(self._id) as conn:
  File "/home/shared/anaconda3/lib/python3.6/multiprocessing/resource_sharer.py", line 87, i


Epoch: 3
loc_loss: 0.097 | cls_loss: 12.262 | train_loss: 12.359 | avg_loss: 12.359 [1/2141]

Test
loc_loss: 0.089 | cls_loss: 12.226 | test_loss: 12.315 | avg_loss: 12.315 [1/2141]
Saving..

Epoch: 4
loc_loss: 0.099 | cls_loss: 12.248 | train_loss: 12.347 | avg_loss: 12.347 [1/2141]

Test
loc_loss: 0.089 | cls_loss: 12.221 | test_loss: 12.310 | avg_loss: 12.310 [1/2141]
Saving..

Epoch: 5
loc_loss: 0.096 | cls_loss: 12.237 | train_loss: 12.333 | avg_loss: 12.333 [1/2141]

Test
loc_loss: 0.089 | cls_loss: 12.213 | test_loss: 12.302 | avg_loss: 12.302 [1/2141]
Saving..


Exception ignored in: <bound method _DataLoaderIter.__del__ of <torch.utils.data.dataloader._DataLoaderIter object at 0x7f33bea74e80>>
Traceback (most recent call last):
  File "/home/shared/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 349, in __del__
    self._shutdown_workers()
  File "/home/shared/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 328, in _shutdown_workers
    self.worker_result_queue.get()
  File "/home/shared/anaconda3/lib/python3.6/multiprocessing/queues.py", line 337, in get
    return _ForkingPickler.loads(res)
  File "/home/shared/anaconda3/lib/python3.6/site-packages/torch/multiprocessing/reductions.py", line 70, in rebuild_storage_fd
    fd = df.detach()
  File "/home/shared/anaconda3/lib/python3.6/multiprocessing/resource_sharer.py", line 57, in detach
    with _resource_sharer.get_connection(self._id) as conn:
  File "/home/shared/anaconda3/lib/python3.6/multiprocessing/resource_sharer.py", line 87, i


Epoch: 6
loc_loss: 0.093 | cls_loss: 12.218 | train_loss: 12.311 | avg_loss: 12.311 [1/2141]

Test


...


Epoch: 180
loc_loss: 0.081 | cls_loss: 5.953 | train_loss: 6.035 | avg_loss: 6.035 [1/2141]

Test
loc_loss: 0.087 | cls_loss: 6.407 | test_loss: 6.494 | avg_loss: 6.494 [1/2141]

Epoch: 181
loc_loss: 0.076 | cls_loss: 6.004 | train_loss: 6.080 | avg_loss: 6.080 [1/2141]

Test
loc_loss: 0.087 | cls_loss: 6.330 | test_loss: 6.417 | avg_loss: 6.417 [1/2141]
Saving..

Epoch: 182
loc_loss: 0.077 | cls_loss: 6.312 | train_loss: 6.390 | avg_loss: 6.390 [1/2141]

Test
loc_loss: 0.087 | cls_loss: 6.288 | test_loss: 6.375 | avg_loss: 6.375 [1/2141]
Saving..

Epoch: 183
loc_loss: 0.080 | cls_loss: 6.101 | train_loss: 6.181 | avg_loss: 6.181 [1/2141]

Test
loc_loss: 0.086 | cls_loss: 6.299 | test_loss: 6.385 | avg_loss: 6.385 [1/2141]

Epoch: 184
loc_loss: 0.089 | cls_loss: 5.236 | train_loss: 5.326 | avg_loss: 5.326 [1/2141]

Test
loc_loss: 0.086 | cls_loss: 6.315 | test_loss: 6.402 | avg_loss: 6.402 [1/2141]

Epoch: 185
loc_loss: 0.085 | cls_loss: 5.563 | train_loss: 5.648 | avg_loss: 5.648 [1/

Exception ignored in: <bound method _DataLoaderIter.__del__ of <torch.utils.data.dataloader._DataLoaderIter object at 0x7f33bea74cf8>>
Traceback (most recent call last):
  File "/home/shared/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 349, in __del__
    self._shutdown_workers()
  File "/home/shared/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 328, in _shutdown_workers
    self.worker_result_queue.get()
  File "/home/shared/anaconda3/lib/python3.6/multiprocessing/queues.py", line 337, in get
    return _ForkingPickler.loads(res)
  File "/home/shared/anaconda3/lib/python3.6/site-packages/torch/multiprocessing/reductions.py", line 70, in rebuild_storage_fd
    fd = df.detach()
  File "/home/shared/anaconda3/lib/python3.6/multiprocessing/resource_sharer.py", line 57, in detach
    with _resource_sharer.get_connection(self._id) as conn:
  File "/home/shared/anaconda3/lib/python3.6/multiprocessing/resource_sharer.py", line 87, i


Epoch: 196
loc_loss: 0.092 | cls_loss: 4.930 | train_loss: 5.022 | avg_loss: 5.022 [1/2141]

Test
loc_loss: 0.087 | cls_loss: 5.949 | test_loss: 6.036 | avg_loss: 6.036 [1/2141]

Epoch: 197
loc_loss: 0.076 | cls_loss: 5.188 | train_loss: 5.265 | avg_loss: 5.265 [1/2141]

Test
loc_loss: 0.087 | cls_loss: 5.983 | test_loss: 6.070 | avg_loss: 6.070 [1/2141]

Epoch: 198
loc_loss: 0.107 | cls_loss: 5.329 | train_loss: 5.436 | avg_loss: 5.436 [1/2141]

Test


Exception ignored in: <bound method _DataLoaderIter.__del__ of <torch.utils.data.dataloader._DataLoaderIter object at 0x7f34284dae10>>
Traceback (most recent call last):
  File "/home/shared/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 349, in __del__
    self._shutdown_workers()
  File "/home/shared/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 328, in _shutdown_workers
    self.worker_result_queue.get()
  File "/home/shared/anaconda3/lib/python3.6/multiprocessing/queues.py", line 337, in get
    return _ForkingPickler.loads(res)
  File "/home/shared/anaconda3/lib/python3.6/site-packages/torch/multiprocessing/reductions.py", line 70, in rebuild_storage_fd
    fd = df.detach()
  File "/home/shared/anaconda3/lib/python3.6/multiprocessing/resource_sharer.py", line 57, in detach
    with _resource_sharer.get_connection(self._id) as conn:
  File "/home/shared/anaconda3/lib/python3.6/multiprocessing/resource_sharer.py", line 87, i

loc_loss: 0.087 | cls_loss: 5.981 | test_loss: 6.068 | avg_loss: 6.068 [1/2141]

Epoch: 199
loc_loss: 0.074 | cls_loss: 6.289 | train_loss: 6.363 | avg_loss: 6.363 [1/2141]

Test
loc_loss: 0.087 | cls_loss: 5.972 | test_loss: 6.059 | avg_loss: 6.059 [1/2141]
